In [1]:
import numpy as np
import scipy
import scipy.io
import scipy.sparse as sp
import surprise
%load_ext autoreload

In [2]:
from data_helpers import *

#Load train and testset for the surprise models
file_path = "../data/data_surprise.csv"
trainset, testset = build_surprise_data(file_path)

#Loads ratings to predict
INPUT_PATH = "../data/sample_submission.csv"
ids = read_csv_sample(INPUT_PATH)

In [3]:
#Load train and testset for the custom models
train, test = split_data(load_data("../data/data_train.csv"), p_test = 0.1)

number of users: 10000, number of items: 1000


In [7]:
from implementations import *

In [8]:
Xtest = []
Xids = []

#Generate predictions with every method

In [ ]:
rmse, Xtest, Xids, preds_test_bsl, preds_ids_bsl = baseline_only(trainset, testset, ids, Xtest, Xids)
rmse, Xtest, Xids, preds_test_kbm, preds_ids_kbm = knn_baseline_movie(trainset, testset, ids, Xtest, Xids)
rmse, Xtest, Xids, preds_test_kbu, preds_ids_kbu = knn_baseline_user(trainset, testset, ids, Xtest, Xids)
rmse, Xtest, Xids, preds_test_svd, preds_ids_svd = svd(trainset, testset, ids, Xtest, Xids)
rmse, Xtest, Xids, preds_test_svdpp, preds_ids_svdpp = svdpp(trainset, testset, ids, Xtest, Xids)
rmse, Xtest, Xids, preds_test, preds_ids = slopeone(trainset, testset, ids, Xtest, Xids)
rmse, Xtest, Xids, preds_test, preds_ids = co_clustering(trainset, testset, ids, Xtest, Xids)
rmse, Xtest, Xids, preds_test, preds_ids = knn_basic_user(trainset, testset,  ids, Xtest, Xids)
rmse, Xtest, Xids, preds_test, preds_ids = knn_basic_movie(trainset, testset,  ids, Xtest, Xids)
rmse, Xtest, Xids, preds_test, preds_ids = knn_centered_user(trainset, testset,  ids, Xtest, Xids)
rmse, Xtest, Xids, preds_test, preds_ids = knn_centered_movie(trainset, testset,  ids, Xtest, Xids)

Baseline Only
   Training RMSE:  0.9898783013438501
   Test RMSE:  0.9988352005327865
kNN Baseline Movie
   Training RMSE:  0.6242593929059024
   Test RMSE:  0.9865931529990231
kNN Baseline User


In [ ]:
rmse, Xtest, Xids, preds_test_als, preds_ids_als = matrix_factorization_als(train, test, ids, Xtest, Xids)
rmse, Xtest, Xids, preds_test_sgd, preds_ids_sgd = matrix_factorization_sgd(train, test, ids, Xtest, Xids)

In [ ]:
np.array(Xtest) , np.array(Xids)

In [ ]:
from sklearn.linear_model import LinearRegression
def blend(preds_test, preds_ids, testset):
    """
    Linear regression that finds the optimal weights of each model
    Argument : preds_test, predicted ratings for the known test set
               preds_ids, predicted ratings for the unknown set
               testset, the testset
    Return : estimations, the final predictions
             weights, coefficients associated to each model
    """
    print('Blending')
    
    #Known ratings of testset
    y_test = [rating for (_,_,rating) in testset]
    
    #Ridge Regression
    linreg = Ridge(alpha=0.1, normalize=True)
    
    #Fit between predicted and know ratings of testset
    linreg.fit(preds_test.T, y_test)
    weights = linreg.coef_
    
    #Predict unknown ratings
    predictions = np.clip(linreg.predict(preds_ids.T), 1, 5)
    
    print(weights, end='\n\n')
    
    #RMSE of regression
    print('Test RMSE: %f' % calculate_rmse(y_test, linreg.predict(preds_test.T)))
    
    #Rounding-off predictions
    estimations = np.zeros(len(predictions))
    for j, pred in enumerate(predictions):
        estimations[j] = round(pred)
        
    return estimations, weights

In [ ]:
#Blending
predictions, weights = blend(np.array(Xtest), np.array(Xids), testset)

In [ ]:
len(predictions)

In [ ]:
from data_helpers import create_csv_submission

OUTPUT_PATH = "../data/submission.csv"
create_csv_submission(ids, predictions, OUTPUT_PATH)
print("File submission.csv ready to be submitted !")